In [1]:
import numpy as np
import cv2
import face_recognition

In [2]:
webcam = cv2.VideoCapture(0, cv2.CAP_DSHOW)    # CAP_DSHOW --> for faster frame rate


while True:
    boolean, img = webcam.read()
    small_img = cv2.resize(img,(0,0),fx=0.25,fy=0.25)
    num_faces = face_recognition.face_locations(small_img, number_of_times_to_upsample = 1, model = "hog")

    for index, location in enumerate(num_faces):
        top, right, bottom, left = location
        top = top * 4
        right = right * 4
        bottom = bottom * 4
        left = left * 4
        
        face_crop = img[top:bottom,left:right]
        
        mean_values = (78.4263377603, 87.7689143744, 114.895847746)  # these are constant values
        binary_large_object = cv2.dnn.blobFromImage(face_crop, 1, (227, 227), mean_values,
                                                    swapRB = False)
        
        gender_label_list = ['Male', 'Female']
        gender_cov_net = cv2.dnn.readNet("gender_net.caffemodel", "gender_deploy.prototxt")
        gender_cov_net.setInput(binary_large_object)
        gender_predictions = gender_cov_net.forward()         #get the predictions from the model
        gender = gender_label_list[gender_predictions[0].argmax()]
        
        
        age_label_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        age_cov_net = cv2.dnn.readNet("age_net.caffemodel", "age_deploy.prototxt")
        age_cov_net.setInput(binary_large_object)
        age_predictions = age_cov_net.forward()
        age = age_label_list[age_predictions[0].argmax()]
        
        
        cv2.rectangle(img, (left,top), (right,bottom), (0,0,200), 2)

        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, gender+" "+age+"yrs", (left,bottom + 15), font, 0.5, (200,0,0),1) 

    cv2.imshow("Faces", img)             # should be outside for loop else won't detect non face images.

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
webcam.release()
cv2.destroyAllWindows() 

-----------